# Gaussion Process Regression Model (Version 1)

Supported by [`sklearn.gaussian_process.GaussianProcessRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) function of ScikitLearn

### Summary

| Techniques                     | Used / Description           |
| ------------------------------ | ---------------------------- |
| Handling Unknown Variables     | Drop Rows                    |
| Handling Categorical Variables | Drop Columns (Drop Features) |
| Handling Class Imbalance       | Not Applied                  |
| Handling Outliers              | Not Applied                  |

### Results

| Metric                 | Value   |
| ---------------------- | ------- |
| RMSE (Lower is better) | 0.85619 |
| R2 (Higher is better)  | 0.43520 |


In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel, DotProduct, ExpSineSquared, RationalQuadratic, Matern
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

In [143]:
X_train = pd.read_csv("../../cleaned-data/X_train.csv")
y_train = pd.read_csv("../../cleaned-data/y_train.csv")

X_test = pd.read_csv("../../cleaned-data/X_test.csv")
y_test = pd.read_csv("../../cleaned-data/y_test.csv")

In [144]:
columns_to_drop = ['land_use_label', 'subzone', 'planning_area', 'region',
                   'temp_2024_04_07_min', 'temp_2024_04_07_max', 'temp_2024_04_07_median',
                   'temp_2024_04_08_min', 'temp_2024_04_08_max', 'temp_2024_04_08_median',
                   'temp_2024_04_09_min', 'temp_2024_04_09_max', 'temp_2024_04_09_median',
                   'temp_2024_04_10_min', 'temp_2024_04_10_max', 'temp_2024_04_10_median']

X_train.drop(columns_to_drop, axis=1, inplace=True)
X_test.drop(columns_to_drop, axis=1, inplace=True)

In [132]:
pd.concat((X_train, y_train)).describe()

,latitude,longitude,distance_to_waterbody,distance_to_open_space,elevation,Total_x,HDB Total,Condominiums & Other Apartments,Landed Properties_x,Other Dwellings_x,...,"$5,000 - $5,999","$6,000 - $6,999","$7,000 - $7,999","$8,000 - $8,999","$9,000 - $9,999","$10,000 - 10,999","$11,000 - 11,999","$12,000 - $14,999","$15,000 & Over",avg_temp
count,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,...,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000
mean,1.333808,103.825289,0.003450,0.003158,18.790507,15881.963993,11266.759411,2835.204583,1632.978723,144.140753,...,5721.201309,4160.641571,3418.569558,2979.972177,2363.749591,2133.001637,1689.857610,3186.022913,5837.605565,30.065189
std,0.033638,0.087219,0.002659,0.005375,11.746306,22527.915515,20687.510550,3817.916875,3707.929779,243.301823,...,4539.120772,3265.876008,2578.008588,2219.346693,1717.486356,1466.083238,1246.052864,2228.565349,4383.833809,1.264064
min,1.260529,103.624030,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.776378
25%,1.311093,103.761419,0.001412,0.000544,11.000000,80.000000,0.000000,0.000000,0.000000,0.000000,...,1799.000000,1239.000000,1138.000000,1250.000000,867.000000,1032.000000,594.000000,1465.000000,4025.000000,28.967618
50%,1.329272,103.832760,0.003000,0.001407,16.000000,8040.000000,0.000000,1110.000000,10.000000,60.000000,...,4562.000000,3488.000000,2753.000000,2367.000000,2038.000000,2030.000000,1422.000000,3180.000000,5440.000000,29.984970
75%,1.355520,103.893404,0.004961,0.002993,25.000000,24320.000000,16685.000000,4910.000000,1085.000000,210.000000,...,10695.000000,7552.000000,5855.000000,5104.000000,3872.000000,3255.000000,2385.000000,4404.000000,7842.000000,30.942114
max,1.429386,103.989652,0.013994,0.041106,91.000000,130980.000000,125230.000000,16920.000000,18850.000000,2730.000000,...,14433.000000,11198.000000,8471.000000,7626.000000,5497.000000,5482.000000,4067.000000,7528.000000,16726.000000,32.836497


In [133]:
valid_rows_train = X_train.min_ndvi != "-"
valid_rows_test = X_test.min_ndvi != "-"

X_train = X_train[valid_rows_train]
y_train = y_train[valid_rows_train]

X_test = X_test[valid_rows_test]
y_test = y_test[valid_rows_test]

In [142]:
kernel = (
    ConstantKernel()
    # * RBF(length_scale=1e-1, length_scale_bounds=(1e-5, 1e5))
    # * Matern(length_scale=1e-1, length_scale_bounds=(1e-5, 1e5), nu=1.5)
    * RationalQuadratic(length_scale=1.0, alpha=1e-1, alpha_bounds=(1e-5, 1e2))
    + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-5, 1e1))
)

pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("gpr", GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)),
    ]
)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('gpr',
                 GaussianProcessRegressor(kernel=1**2 * RationalQuadratic(alpha=0.1, length_scale=1) + WhiteKernel(noise_level=0.01),
                                          n_restarts_optimizer=10))])

In [141]:
y_mean, y_std = pipeline.predict(X_test, return_std=True)

rmse = mean_squared_error(y_test, y_mean, squared=False)
r2 = r2_score(y_test, y_mean)

print(f"STD: {np.mean(y_std)}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

STD: 0.9924402808904845
RMSE: 0.8561996709616964
R2: 0.4352058597107318
